In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipol_embedd_7410_128_768_alberto_training.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipolc_wmal_training_7410_128.joblib")

In [ ]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/sentipolc_train_processed.csv")

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["oneg"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 0]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]
[1]


In [ ]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate, Dropout
!pip install keras-self-attention
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention
from keras.layers import Add, Multiply

In [ ]:
import joblib
import pandas as pd
emb_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipol_embedd_2000_128_768_alberto_test.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/sentipolc_test_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipolc_wmal_test_2000_128.joblib")

In [ ]:
# define example
data = dataset_test["oneg"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
print(onehot_encoded_test)

[0 0 1 ... 1 1 0]
[0 0 1 ... 1 1 0]
[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [ ]:
import keras
from sklearn.metrics import precision_recall_fscore_support as score
import numpy as np

input_emb = Input(shape=(128,768))
input_wmal = Input(shape=(128,))

#Bi-LSTM
#bi = Bidirectional(LSTM(64, return_sequences=True))(input_emb)
bi = LSTM(64, return_sequences=True)(input_emb)
att = SeqSelfAttention(attention_width=15,attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       attention_activation="sigmoid",
                       kernel_regularizer=keras.regularizers.l2(1e-8),#(1e-6),
                       bias_regularizer=keras.regularizers.l1(1e-4),#(1e-4),
                       attention_regularizer_weight=1e-8,#1e-4,
                       use_attention_bias=True) (bi)

app = Add() ([bi, att])                      
flat = Flatten()(app)
den1 = Dense(4000) (flat)
den2 = Dense(2000) (den1)
den3 = Dense(1000) (den2)
den4 = Dense(500) (den3)
den5 = Dense(256) (den4)
den6 = Dense(128) (den5)
den7 = Dense(64) (den6)

#WMAL 
d1 = Dense(64) (input_wmal)

#Concat
concat = Concatenate(axis=1)([den7, d1])
d1 = Dense(32, activation="relu") (concat)
output = Dense(2,activation="softmax") (d1)

m = keras.Model(inputs=[input_emb,input_wmal], outputs=output)

opt = keras.optimizers.RMSprop()

m.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

class myCallback(keras.callbacks.Callback):
    bestF1=0
    def __init__(self):
        super().__init__()
        self._get_pred = None
        self.preds = []
    def on_epoch_end(self,epoch, logs=None):
        predictions = m.predict([emb_matrix_test,wmal_matrix_test])
        predictions_vec = np.zeros((2000,4))
        i = 0
        confidences= []
        for item in predictions:
          index = np.argmax(item);
          confidences.append(item[index])
          predictions_vec[i][index] = 1
          i = i+1
        labels_to_eval = []
        for vec in predictions_vec:
          inverted = label_encoder.inverse_transform([argmax(vec)])[0]
          labels_to_eval.append(inverted)
        
        precision,recall,fscore,support=score(dataset_test["oneg"],labels_to_eval,average='macro')
        print("Test f1-score:"+str(fscore))
        if fscore > self.bestF1:
          m.save("bestModel.h5")
          print("BEST MODEL!!!!!")
          self.bestF1 = fscore

callbacks = [myCallback()]

In [ ]:
m.summary()

In [ ]:
m.fit([emb_matrix_train,wmal_matrix_train],onehot_encoded,batch_size=128, epochs=200,validation_split=0.15, callbacks=[callbacks])
#m.load_weights("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_3_sentipolc/model_3_neg_sentipolc_0.72803.h5")

Epoch 1/200
50/50 [==============================] - 5s 62ms/step - loss: 376.2514 - accuracy: 0.5343 - val_loss: 3.1764 - val_accuracy: 0.7059
Test f1-score:0.6322810572530617
BEST MODEL!!!!!
Epoch 2/200
50/50 [==============================] - 3s 53ms/step - loss: 3.5906 - accuracy: 0.5668 - val_loss: 0.4498 - val_accuracy: 0.8255
Test f1-score:0.38644358221576525
Epoch 3/200
50/50 [==============================] - 3s 54ms/step - loss: 54.4928 - accuracy: 0.5573 - val_loss: 0.6552 - val_accuracy: 0.7491
Test f1-score:0.5552431098549518
Epoch 4/200
50/50 [==============================] - 3s 53ms/step - loss: 7.4534 - accuracy: 0.5886 - val_loss: 0.6275 - val_accuracy: 0.8165
Test f1-score:0.40196641950834394
Epoch 5/200
50/50 [==============================] - 3s 53ms/step - loss: 0.6788 - accuracy: 0.5723 - val_loss: 0.7190 - val_accuracy: 0.5360
Test f1-score:0.654307464933432
BEST MODEL!!!!!
Epoch 6/200
50/50 [==============================] - 3s 54ms/step - loss: 33.1054 - accur

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


50/50 [==============================] - 3s 53ms/step - loss: 31.2324 - accuracy: 0.5332 - val_loss: 0.6732 - val_accuracy: 0.8273
Test f1-score:0.3806131929389905
Epoch 10/200
50/50 [==============================] - 3s 53ms/step - loss: 3.3559 - accuracy: 0.5494 - val_loss: 0.7528 - val_accuracy: 0.8291
Test f1-score:0.3806131929389905
Epoch 11/200
50/50 [==============================] - 3s 53ms/step - loss: 0.6987 - accuracy: 0.5546 - val_loss: 0.6480 - val_accuracy: 0.8327
Test f1-score:0.38080495356037153
Epoch 12/200
50/50 [==============================] - 3s 53ms/step - loss: 0.6880 - accuracy: 0.5531 - val_loss: 0.6397 - val_accuracy: 0.8327
Test f1-score:0.38080495356037153
Epoch 13/200
50/50 [==============================] - 3s 54ms/step - loss: 0.6884 - accuracy: 0.5492 - val_loss: 0.6337 - val_accuracy: 0.8327
Test f1-score:0.38080495356037153
Epoch 14/200
50/50 [==============================] - 3s 53ms/step - loss: 0.6870 - accuracy: 0.5561 - val_loss: 0.6292 - val_acc

In [ ]:
m.load_weights("bestModel.h5")
predictions = m.predict([emb_matrix_test,wmal_matrix_test])

In [ ]:
import numpy as np
predictions_vec = np.zeros((2000,4))
i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1
labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [ ]:
!pip install scikit-learn

from sklearn.metrics import classification_report
print(classification_report(dataset_test["oneg"],labels_to_eval,digits=5))

              precision    recall  f1-score   support

           0    0.71694   0.81545   0.76303      1230
           1    0.62230   0.48571   0.54559       770

    accuracy                        0.68850      2000
   macro avg    0.66962   0.65058   0.65431      2000
weighted avg    0.68050   0.68850   0.67931      2000



In [ ]:
m.save("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_new_neg_sentipolc_0.74049.h5")

In [ ]:
dataset_test.to_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/predictions_sentipolc_neg_full_0.74049.csv");